In [18]:
# nuclio: ignore
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter
import nuclio

In [19]:
%%nuclio env
USER=sam
VERSION=1.0

%nuclio: setting 'USER' environment variable
%nuclio: setting 'VERSION' environment variable


In [20]:
%nuclio env PASSWORD=${PASSWORD}

%nuclio: setting 'PASSWORD' environment variable


In [21]:
%nuclio config spec.build.baseImage = "python:3.11"
%nuclio config spec.runtime = "python:3.11"

%nuclio: setting spec.build.baseImage to 'python:3.11'
%nuclio: setting spec.runtime to 'python:3.11'


In [23]:
%%nuclio cmd
pip install requests
pip install msgpack
pip install numpy
pip install opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 3.4 MB/s eta 0:00:00a 0:00:01
  Using cached opencv_python-4.11.0.86-cp37-abi3-macosx_13_0_arm64.whl.metadata (20 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-macosx_13_0_arm64.whl (37.3 MB)


In [41]:
import os 
import base64
import json
from brightness_detector import calculate_brightness

def handler(context, event):
    """
    Nuclio handler function for brightness detection
    """
    try:
        # Get headers and body
        headers = event.headers
        body = event.body
        
        # Process based on content type
        content_type = headers.get('content-type', '')
        
        if event.method != 'POST':
            return {
                'statusCode': 400,
                'body': json.dumps({'error': 'Only POST requests are supported'})
            }
        
        # Parse ROI if specified
        roi = None
        roi_param = event.fields.get('roi')
        if roi_param:
            try:
                roi = tuple(map(int, roi_param.split(',')))
                if len(roi) != 4:
                    raise ValueError()
            except:
                return {
                    'statusCode': 400,
                    'body': json.dumps({'error': 'Invalid ROI format. Expected x,y,width,height'})
                }
        
        # Handle different content types
        if 'multipart/form-data' in content_type:
            # Get the image file from multipart form
            file_data = event.fields.get('image')
            if file_data:
                # Decode base64 if needed
                if isinstance(file_data, str) and file_data.startswith('data:image'):
                    # Handle data URI
                    _, file_data = file_data.split(',', 1)
                    image_data = base64.b64decode(file_data)
                elif isinstance(file_data, dict) and 'data' in file_data:
                    # Handle Nuclio's multipart format
                    image_data = file_data['data']
                else:
                    image_data = file_data
                
                # Calculate brightness
                brightness = calculate_brightness(image_data=image_data, roi=roi)
                
                # Return the brightness value
                return {
                    'statusCode': 200,
                    'body': json.dumps({'brightness': brightness}),
                    'content_type': 'application/json'
                }
            else:
                return {
                    'statusCode': 400,
                    'body': json.dumps({'error': 'No image file found in request'})
                }
                
        elif 'application/json' in content_type:
            # Parse JSON body
            try:
                data = json.loads(body)
                
                # Check if image path is provided
                if 'image_path' in data:
                    # Use the provided path - be careful with this for security reasons
                    image_path = data['image_path']
                    roi = data.get('roi')
                    brightness = calculate_brightness(image_path=image_path, roi=roi)
                    return {
                        'statusCode': 200,
                        'body': json.dumps({'brightness': brightness}),
                        'content_type': 'application/json'
                    }
                    
                # Check if base64 image is provided
                elif 'image_base64' in data:
                    # Decode base64 image
                    image_data = base64.b64decode(data['image_base64'])
                    brightness = calculate_brightness(image_data=image_data, roi=roi)
                    return {
                        'statusCode': 200,
                        'body': json.dumps({'brightness': brightness}),
                        'content_type': 'application/json'
                    }
                else:
                    return {
                        'statusCode': 400,
                        'body': json.dumps({'error': 'No image data found in request'})
                    }
            except Exception as e:
                return {
                    'statusCode': 400,
                    'body': json.dumps({'error': f'Error parsing JSON: {str(e)}'})
                }
        else:
            # Handle raw image data
            try:
                brightness = calculate_brightness(image_data=body, roi=roi)
                return {
                    'statusCode': 200,
                    'body': json.dumps({'brightness': brightness}),
                    'content_type': 'application/json'
                }
            except Exception as e:
                return {
                    'statusCode': 400,
                    'body': json.dumps({'error': f'Error processing image: {str(e)}'})
                }
                
    except Exception as e:
        context.logger.error(f'Error: {str(e)}')
        return {
            'statusCode': 500,
            'body': json.dumps({'error': str(e)}),
            'content_type': 'application/json'
        }


In [43]:
# nuclio: ignore
import nuclio
import json

# Explicitly set method='POST' in the Event object
event = nuclio.Event(
    method='POST',  # Add this line to fix the issue
    body=json.dumps({
        'image_path': 'test/test_light_off.jpg', 
        'roi': [640, 360, 640, 360]
    }),
    headers={'content-type': 'application/json'}
)

# Test the handler
result = handler(context, event)
print(result)

{'statusCode': 200, 'body': '{"brightness": 59.65424045138889}', 'content_type': 'application/json'}


In [44]:
%%nuclio markdownblock
# @include-import-file:brightness_detector.py
%nuclio deploy -n fn_get_brightness -p avi -d http://lasso:8070

[nuclio] 2025-03-28 16:27:38,214 (info) Staging files and preparing base images
[nuclio] 2025-03-28 16:27:38,216 (warn) Using user provided base image, runtime interpreter version is provided by the base image
[nuclio] 2025-03-28 16:27:38,218 (info) Building processor image
[nuclio] 2025-03-28 16:27:49,574 (info) Build complete
[nuclio] 2025-03-28 16:27:51,200 Failed to deploy. Details:

Error - Container exited with status: 2
    /nuclio/pkg/dockerclient/shell.go:580

Call stack:
Container exited with status: 2
    /nuclio/pkg/dockerclient/shell.go:580
Container 4a17437353478ce2da93ce8a20f4a2a89c15fb27acc85a52c9c8e03b5e4d3d70 is not healthy
    /nuclio/pkg/dockerclient/shell.go:612
Function wasn't ready in time. Logs:
25.03.28 15:27:50.430 (I)                 processor Starting processor {"version": "Label: 1.14.1, Git commit: f7573701a7cd7f093fe37481bd8e61d6979a3d14, OS: linux, Arch: amd64, Go version: go1.21.13"}
25.03.28 15:27:50.431 (D)                 processor Read configuration

%nuclio: error: cannot deploy 


In [9]:
!curl lasso:32771

Hello World from VS codesam

In [10]:
%nuclio show

%nuclio: notebook nuclio-example exported
Config:
apiVersion: nuclio.io/v1
kind: Function
metadata:
  annotations:
    nuclio.io/generated_by: function generated from /Users/raffael/octanis/umami/avi_fn_get_brightness/nuclio-example.ipynb
  labels: {}
  name: nuclio-example
spec:
  build:
    baseImage: python:3.11
    commands:
    - pip install requests
    - pip install msgpack
    functionSourceCode: IyBHZW5lcmF0ZWQgYnkgbnVjbGlvLmV4cG9ydC5OdWNsaW9FeHBvcnRlcgoKZnJvbSBvcyBpbXBvcnQgZW52aXJvbgpkZWYgaGFuZGxlcihjb250ZXh0LCBldmVudCk6CiAgICBjb250ZXh0LmxvZ2dlci5pbmZvKCdzb21lIGxvZyBtZXNzYWdlJykKICAgIHJldHVybiAnSGVsbG8gV29ybGQgZnJvbSBWUyBjb2RlJytlbnZpcm9uLmdldCgnVVNFUicpCgo=
    noBaseImagesPull: true
  env:
  - name: USER
    value: sam
  - name: VERSION
    value: '1.0'
  - name: PASSWORD
    value: ${PASSWORD}
  handler: nuclio-example-nuclio:handler
  runtime: python
  volumes: []

Code:
# Generated by nuclio.export.NuclioExporter

from os import environ
def handler(context, event):
    c

In [48]:
%nuclio nuclio.print_handler_code

%nuclio: unknown command: nuclio.print_handler_code
